# SBERT Encoder w/ MLP MoE for Multilabel Film Genre Classification

Short test poses a unique challenge for many NLP models due to limited context length while retaining the difficulties of the nuances of structured languages. This model aims to use several methods to address each of these concerns individually. First, a trained autoencoder, SBERT, will be used to create a new hidden representation for the short text input sequences. Then for the diverse classification tasks several expert models will be trained to selectively specialize in specific genre embeddings. Simple MLP models are sufficient for mapping the smaller embeddings to output classes while keeping the full model simple to train.

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import lightning.pytorch as pl
from lightning.pytorch.callbacks import BaseFinetuning
from torchmetrics.classification import MultilabelF1Score
from sentence_transformers import SentenceTransformer

# ==========================================
# 1. MODEL ARCHITECTURE (MoE Components)
# ==========================================

class Expert(nn.Module):
    """
    A simple Feed-Forward Network acting as a single 'Expert'.
    """
    def __init__(self, input_dim, hidden_dim, output_dim, dropout=0.1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

class TopKRouter(nn.Module):
    """
    Gating Network that selects the top-k experts for each input.
    """
    def __init__(self, input_dim, num_experts, top_k=2):
        super().__init__()
        self.gate = nn.Linear(input_dim, num_experts)
        self.top_k = top_k

    def forward(self, x):
        logits = self.gate(x)
        top_k_vals, top_k_indices = torch.topk(logits, self.top_k, dim=1)
        router_probs = F.softmax(top_k_vals, dim=1)
        return router_probs, top_k_indices, logits

class MoEClassifier(nn.Module):
    """
    The Mixture of Experts Classification Head.
    """
    def __init__(self, input_dim, num_classes, num_experts=4, top_k=2, expert_hidden_dim=128):
        super().__init__()
        self.num_experts = num_experts
        self.top_k = top_k
        
        self.router = TopKRouter(input_dim, num_experts, top_k=top_k)
        self.experts = nn.ModuleList([
            Expert(input_dim, expert_hidden_dim, num_classes) 
            for _ in range(num_experts)
        ])

    def forward(self, x):
        batch_size = x.size(0)
        router_probs, expert_indices, router_logits = self.router(x)
        
        final_output = torch.zeros(batch_size, self.experts[0].net[-1].out_features).to(x.device)
        
        for k in range(self.top_k):
            selected_experts = expert_indices[:, k]
            gate_weight = router_probs[:, k].unsqueeze(1)
            
            for expert_idx in range(self.num_experts):
                mask = (selected_experts == expert_idx)
                if mask.any():
                    expert_input = x[mask]
                    expert_output = self.experts[expert_idx](expert_input)
                    final_output[mask] += gate_weight[mask] * expert_output
                    
        return final_output, router_logits

class SBERT_MoE_Model(nn.Module):
    """
    Full End-to-End Model: SBERT Encoder + MoE Classifier
    """
    def __init__(self, model_name='all-MiniLM-L6-v2', num_classes=5, num_experts=8, top_k=2):
        super().__init__()
        # 1. Initialize SBERT
        self.sbert = SentenceTransformer(model_name)
        
        # 2. Capture device choice
        target_device = self.sbert.device 
        
        # 3. Initialize MoE head
        embedding_dim = self.sbert.get_sentence_embedding_dimension()
        self.moe_head = MoEClassifier(
            input_dim=embedding_dim,
            num_classes=num_classes,
            num_experts=num_experts,
            top_k=top_k
        )
        
        # 4. Force MoE head to same device
        self.moe_head.to(target_device)

    def forward(self, text_input):
        # Encode and Detach to treat embeddings as fixed features initially
        features = self.sbert.encode(text_input, convert_to_tensor=True)
        
        # Important: Clone and detach to avoid "Inference Tensor" errors during backprop
        # If unfreezing later, the optimizer handles the graph connection, 
        # but for the initial forward pass logic, this is safe.
        features = features.clone().detach() 
        
        logits, router_logits = self.moe_head(features)
        return logits, router_logits

# ==========================================
# 2. LIGHTNING MODULE & CALLBACKS
# ==========================================

class MoE_LightningModule(pl.LightningModule):
    def __init__(self, model, num_classes, num_experts, learning_rate=1e-3, aux_loss_weight=0.1):
        super().__init__()
        self.save_hyperparameters(ignore=['model'])
        
        # Organize modules
        self.backbone = model.sbert
        self.head = model.moe_head
        self.learning_rate = learning_rate
        self.num_experts = num_experts
        self.aux_loss_weight = aux_loss_weight
        
        # Multi-Label Loss & Metrics
        self.criterion = nn.BCEWithLogitsLoss()
        self.val_f1 = MultilabelF1Score(num_labels=num_classes, threshold=0.5, average='micro')

    def forward(self, x):
        # 1. Tokenize the raw text
        # This converts ["Hello world"] into {'input_ids': ..., 'attention_mask': ...}
        features = self.backbone.tokenize(x)
        
        # 2. Move inputs to the correct device (GPU/MPS)
        # LightningModule provides self.device
        features = {key: value.to(self.device) for key, value in features.items()}
        
        # 3. Pass through SBERT backbone
        # calling .forward() or __call__() allows gradients to flow (unlike .encode())
        out = self.backbone(features)
        
        # 4. Extract the sentence embedding
        embeddings = out['sentence_embedding']
        
        # 5. Pass through MoE Head
        return self.head(embeddings)

    def _compute_load_balancing_loss(self, router_logits):
        probs = F.softmax(router_logits, dim=1)
        mean_probs = probs.mean(dim=0)
        aux_loss = (mean_probs ** 2).sum() * self.num_experts
        return aux_loss

    def training_step(self, batch, batch_idx):
        texts, targets = batch
        logits, router_logits = self(texts)
        
        cls_loss = self.criterion(logits, targets)
        aux_loss = self._compute_load_balancing_loss(router_logits)
        
        total_loss = cls_loss + (self.aux_loss_weight * aux_loss)
        
        self.log("train_loss", total_loss)
        return total_loss

    def validation_step(self, batch, batch_idx):
        texts, targets = batch
        
        # Forward pass (we don't need router logits for validation metrics)
        logits, _ = self(texts)
        
        # Calculate Loss
        val_loss = self.criterion(logits, targets)
        
        # Update F1 Score
        self.val_f1(logits, targets)
        
        # Log metrics so the Scheduler can find 'val_loss'
        self.log("val_loss", val_loss, prog_bar=True)
        self.log("val_f1", self.val_f1, prog_bar=True)

    # def configure_optimizers(self):
    #     # Filter ensures we don't crash trying to optimize frozen params
    #     return torch.optim.AdamW(filter(lambda p: p.requires_grad, self.parameters()), lr=self.learning_rate)
    
    def configure_optimizers(self):
        # 1. Initialize Optimizer with ONLY the Head (Backbone added later by Callback)
        optimizer = torch.optim.AdamW(
            self.head.parameters(), 
            lr=1e-3, 
            weight_decay=0.01
        )
        
        # 2. Use ReduceLROnPlateau (Safe for dynamic unfreezing)
        # It waits for 'val_loss' to stop improving, then lowers LR
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.1,
            patience=2
        )
        
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "monitor": "val_loss", # Required for ReduceLROnPlateau
                "interval": "epoch",
                "frequency": 1
            }
        }
        
class ProductionFinetuning(BaseFinetuning):
    def __init__(self, unfreeze_at_epoch=2, backbone_lr=2e-5):
        super().__init__()
        self._unfreeze_at_epoch = unfreeze_at_epoch
        self._backbone_lr = backbone_lr

    def freeze_before_training(self, pl_module):
        # Start with SBERT frozen
        self.freeze(pl_module.backbone)

    def finetune_function(self, pl_module, current_epoch, optimizer):
        # When we hit the target epoch...
        if current_epoch == self._unfreeze_at_epoch:
            print(f"\n❄️ -> 🔥 Unfreezing SBERT Backbone at Epoch {current_epoch}")
            
            # Unfreeze AND add to the optimizer with the Low LR
            self.unfreeze_and_add_param_group(
                modules=pl_module.backbone,
                optimizer=optimizer,
                lr=self._backbone_lr 
            )

class SBERTUnfreezeCallback(BaseFinetuning):
    def __init__(self, unfreeze_at_epoch=2):
        super().__init__()
        self._unfreeze_at_epoch = unfreeze_at_epoch

    def freeze_before_training(self, pl_module):
        self.freeze(pl_module.backbone)

    def finetune_function(self, pl_module, current_epoch, optimizer):
        if current_epoch == self._unfreeze_at_epoch:
            print(f"\n❄️ -> 🔥 Unfreezing SBERT Backbone at Epoch {current_epoch}")
            self.unfreeze_and_add_param_group(
                modules=pl_module.backbone,
                optimizer=optimizer,
                lr=pl_module.learning_rate / 10.0 
            )

# ==========================================
# 3. DATASET & UTILS
# ==========================================

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels 
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

def map_logits_to_labels(logits, class_names, threshold=0.5):
    probs = torch.sigmoid(logits).cpu()
    predictions_mask = (probs > threshold).numpy()
    batch_results = []
    for is_active in predictions_mask:
        active_labels = [class_names[idx] for idx, val in enumerate(is_active) if val]
        batch_results.append(active_labels)
    return batch_results, probs

# ==========================================
# 4. MAIN EXECUTION
# ==========================================

# --- Setup Data ---
# Example: 4 Classes
CLASS_NAMES = ["Finance", "Sports", "Tech", "Urgent"]

texts = [
    # 1. Finance (Market Report)
    "Inflation data released this morning caused immediate volatility across global markets. The S&P 500 dropped 2% within minutes of the opening bell as traders reacted to the Federal Reserve's signaling of potentially higher interest rates for the remainder of Q4.",
    
    # 2. Tech + Sports (Wearable Tech)
    "The new smart-jersey utilizes embedded bio-sensors to track player fatigue in real-time during the match. Coaches can now monitor heart rate variability and sprint speed directly from the sidelines, allowing for data-driven substitution decisions in the fourth quarter.",
    
    # 3. Sports + Finance (Contract News)
    "After months of negotiation, the star quarterback has agreed to a record-breaking 50 million dollar extension. This deal makes him the highest-paid player in league history and significantly impacts the team's salary cap for the upcoming trading season.",
    
    # 4. Tech (Product Review)
    "Benchmark tests for the new M3 processor show a 15% drop in multi-core performance compared to the previous generation. Thermal throttling appears to be the primary bottleneck, as the chip reaches 90 degrees Celsius under sustained heavy workloads like video rendering.",
    
    # 5. Tech + Urgent (Server Incident)
    "CRITICAL ALERT: Primary database cluster US-East-1 is currently unresponsive due to a cascading failure in the load balancer. Response times have spiked to 5000ms. Immediate engineering intervention is required to prevent a total service outage for enterprise customers.",
    
    # 6. Finance + Urgent (Budget Crisis)
    "Emergency board meeting required: The Q3 projections show a cash flow deficit that will impact payroll by next Friday. We need to approve the emergency line of credit immediately to ensure operations continue without interruption."
]

# Multi-hot labels (Float Tensors)
labels = torch.tensor([
    [1.0, 0.0, 0.0, 0.0], # Finance
    [0.0, 1.0, 1.0, 0.0], # Tech + Sports
    [1.0, 1.0, 0.0, 0.0], # Finance + Sports
    [0.0, 0.0, 1.0, 0.0], # Tech
    [0.0, 0.0, 1.0, 1.0], # Tech + Urgent
    [1.0, 0.0, 0.0, 1.0], # Finance + Urgent
])

dataset = TextDataset(texts, labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# --- Initialize Models ---
num_classes = len(CLASS_NAMES)
backbone = SBERT_MoE_Model(num_classes=num_classes, num_experts=4, top_k=2)

# Wrap in Lightning
pl_module = MoE_LightningModule(
    model=backbone, 
    num_classes=num_classes, 
    num_experts=4,
    aux_loss_weight=0.1
)

# Callback to unfreeze SBERT after 1 epoch
# unfreeze_cb = SBERTUnfreezeCallback(unfreeze_at_epoch=1)
finetune_cb = ProductionFinetuning(unfreeze_at_epoch=3, backbone_lr=2e-5)

# --- Train ---
print("--- Starting Training ---")
trainer = pl.Trainer(
    max_epochs=15,
    accelerator="auto",
    callbacks=[finetune_cb],
    enable_progress_bar=True,
    log_every_n_steps=1 
)

# trainer.fit(pl_module, dataloader, dataloader)
trainer.fit(
    pl_module, 
    train_dataloaders=dataloader, 
    val_dataloaders=dataloader # Using same data for demo purposes
)

# --- Inference ---
print("\n--- Running Inference ---")
pl_module.eval()

test_texts = [
    "The tech company bought a sports team.", 
    "Urgent: The bank is collapsing."
]

with torch.no_grad():
    # Run forward pass
    logits, _ = pl_module(test_texts)
    
    # Map outputs
    predicted_labels, probs = map_logits_to_labels(logits, CLASS_NAMES, threshold=0.5)
    
    for text, lbls, raw_probs in zip(test_texts, predicted_labels, probs):
        print(f"\nInput: {text}")
        print(f"Predicted Labels: {lbls}")
        print(f"Raw Probabilities: {[round(p, 2) for p in raw_probs.tolist()]}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores

  | Name      | Type                | Params | Mode 
----------------------------------------------------------
0 | backbone  | SentenceTransformer | 22.7 M | train
1 | head      | MoEClassifier       | 200 K  | train
2 | criterion | BCEWithLogitsLoss   | 0      | train
3 | val_f1    | MultilabelF1Score   | 0      | train
----------------------------------------------------------
200 K     Trainable params
22.7 M    Non-trainable params
22.9 M    Total params
91.656    Total estimated model params size (MB)
34        Modules in train mode
120       Modules in eval mode


--- Starting Training ---
Epoch 2: 100%|██████████| 3/3 [00:00<00:00, 12.97it/s, v_num=11, val_loss=0.676, val_f1=0.762]
❄️ -> 🔥 Unfreezing SBERT Backbone at Epoch 3
Epoch 14: 100%|██████████| 3/3 [00:00<00:00,  8.03it/s, v_num=11, val_loss=0.514, val_f1=1.000]

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 3/3 [00:00<00:00,  3.61it/s, v_num=11, val_loss=0.514, val_f1=1.000]

--- Running Inference ---

Input: The tech company bought a sports team.
Predicted Labels: ['Finance', 'Sports']
Raw Probabilities: [0.5, 0.55, 0.47, 0.39]

Input: Urgent: The bank is collapsing.
Predicted Labels: ['Finance', 'Urgent']
Raw Probabilities: [0.54, 0.43, 0.49, 0.57]
